Cambiare i Path in base alle locazioni dei file desiderati

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128
IMAGE_SIZE = [96, 96]

#### Test model on Test Set 

In [27]:
import tensorflow as tf
from functools import partial

FILENAMES1=tf.io.gfile.glob("/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group6/" + "*.tfrecords" )
FILENAMES2= tf.io.gfile.glob("/content/drive/MyDrive/Final_Artificial_Vision/dataset_tf_parts/group7/" + "*.tfrecords" )

FILENAMES=FILENAMES1 + FILENAMES2

TEST_FILENAMES = FILENAMES

print("Test TFRecord Files:", len(TEST_FILENAMES))

Test TFRecord Files: 50


In [28]:

def decode_image(image_raw):
    image_shape = tf.stack([96, 96, 3])
    image = tf.io.decode_raw(image_raw, tf.uint8)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, image_shape)
    
    return image

def read_tfrecord(example, labeled):
    tf.compat.v1.enable_eager_execution()
    tfrecord_format = (
        {
            "label_regr": tf.io.FixedLenFeature([],tf.float32),
            "label_class": tf.io.FixedLenFeature([101],tf.int64),
            "label_order" : tf.io.FixedLenFeature([101],tf.int64),
            "image" : tf.io.FixedLenFeature([],tf.string),
        }
        if labeled
        else {"image": tf.io.FixedLenFeature([], tf.string),}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])
    if labeled: 
        label = example["label_class"]
        return image, label
    return image
 

In [29]:

def load_dataset(filenames, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset


In [30]:

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames, labeled=labeled)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset


In [ ]:
from google.colab.patches import cv2_imshow

def show_batch(image_batch,label_batch):
    count=0
    for n in range(0,len(image_batch)):
        cv2_imshow(image_batch[n])
        print("Label:" + str(label_batch[n])) 

test_dataset = get_dataset(TEST_FILENAMES, labeled=True)

count=0
for image_batch,label_batch in iter(test_dataset):
  count=count+len(image_batch)
  print(str(count) + " di 1014120")
print("Number of images for Test: " + str(count))


In [ ]:
## Show image and label of single batch in Test Set 

image_batch,label_batch= next(iter(test_dataset))
show_batch(image_batch.numpy(),label_batch.numpy())

In [40]:
## load model 
from tensorflow import keras

model=keras.models.load_model("/content/drive/MyDrive/Final_Artificial_Vision/CNN/vggFace_classificatore_full1.h5") 

In [41]:
# CUSTOM METRIC

import keras.backend as K
from tensorflow.keras.metrics import mean_absolute_error

def mae(y_true, y_pred):

  sum=0
  y_true = K.cast(y_true, y_pred.dtype)

  ages_true = tf.map_fn(lambda true: K.argmax(true), y_true, dtype=tf.int64)
  ages_pred = tf.map_fn(lambda pred: K.argmax(pred), y_pred, dtype=tf.int64)

  for i in range(0,len(ages_true)):
    err=abs(ages_true[i]-ages_pred[i])
    sum=sum+err
  
  return sum

In [ ]:
#Test our model

count=0
sum=0
for image_batch,label_batch in iter(test_dataset):
  count=count+len(image_batch)
  #ypred.extend(model.predict(image_batch))
  sum=sum+mae(label_batch,model.predict(image_batch))
  print(str(count) + " di 1014120")
  #tf.print(mae_temp)
  print("MAE " + str(count)+ " di 1014120 : ")
  tf.print(sum/count)
print("FINAL MAE: ")
tf.print(sum/count)




#### Create CSV on Test images 

To create a csv you must pass image as TFRecords 


In [65]:
import tensorflow as tf
from functools import partial

TEST_FILENAMES = tf.io.gfile.glob("/content/drive/MyDrive/Final_Artificial_Vision/testset/" + "*.tfrecords" )

print("Test TFRecord Files:", len(TEST_FILENAMES))

Test TFRecord Files: 4


In [66]:
import tensorflow as tf
import keras.backend as K
import numpy as np 

def read_tfrecord_test(example,labeled):
    tfrecord_format = (
        {
            "image" : tf.io.FixedLenFeature([],tf.string),
            "relative_path" : tf.io.FixedLenFeature([],tf.string),
        }
        if labeled
        else {"image": tf.io.FixedLenFeature([], tf.string),}
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])
    path=example["relative_path"]
    return image,path


In [67]:

def load_dataset(filenames, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord_test, labeled=labeled), num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset


In [ ]:
from google.colab.patches import cv2_imshow

test_dataset = get_dataset(TEST_FILENAMES, labeled=True)

count=0
for image_batch,path_batch in iter(test_dataset):
  count=count+len(image_batch)
  print(str(count) + " di 168872")
print("Number of Test Images: " + str(count))


In [69]:
def show_batch(image_batch,path_batch):
    count=0
    for n in range(0,len(image_batch)):
        cv2_imshow(image_batch[n])
        print("Path:" + str(path_batch[n].decode()))


In [ ]:
image_batch,path_batch= next(iter(test_dataset))
show_batch(image_batch.numpy(),path_batch.numpy())

In [71]:
model=keras.models.load_model("/content/drive/MyDrive/Final_Artificial_Vision/CNN/vggFace_classificatore_full1.h5")  

In [75]:
import csv

def csv_result(path,label):
    with open('result_VGGFace2.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([path, label])


In [76]:
def create_csv(predict_batch,path_batch):

  for i in range(0,len(path_batch)):
    label=predict_batch[i].argmax(axis=0)
    path=path_batch[i].numpy().decode()
    csv_result(path,label)



In [ ]:
count=0
for image_batch,path_batch in iter(test_dataset):
  count=count+len(image_batch)
  create_csv(model.predict(image_batch),path_batch)
  print(str(count) + " di 168872" )

print(" Number of images saved: " + str(count))